In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_206666/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
%cd ../

/home/nikita/ML/work_repo/vtb_competition/vtb_data_fusion_contest


In [3]:
from glob import glob

import numpy as np
import pandas as pd
import torch

import pytorch_lightning as pl

from pyhocon import ConfigFactory

In [4]:
FOLD_ID = 1

fold_id_test = FOLD_ID

folds_count = len(glob('data/train_matching_*.csv'))
folds_count

6

In [5]:
# fold_id_valid = np.random.choice([i for i in range(folds_count) if i != fold_id_test], size=1)[0]
fold_id_valid = (fold_id_test + 1) % folds_count
fold_id_valid

2

In [6]:
df_matching_train = pd.concat([pd.read_csv(f'data/train_matching_{i}.csv')
                              for i in range(folds_count) 
                              if i not in (fold_id_test, fold_id_valid)])
df_matching_valid = pd.read_csv(f'data/train_matching_{fold_id_valid}.csv')
df_matching_test = pd.read_csv(f'data/train_matching_{fold_id_test}.csv')

[len(df) for df in [df_matching_train, df_matching_valid, df_matching_test]]

[11721, 2930, 2930]

In [7]:
def click_types(df):
    df['cat_id'] = df['cat_id'].astype(str)
    return df[['user_id', 'timestamp', 'cat_id']]

def click_pivot(df):
    clickstream_embed = df.pivot_table(index = 'user_id', 
                            values=['timestamp'],
                            columns=['cat_id'],
                            aggfunc=['count']).fillna(0)
    clickstream_embed.columns = [f'click-{str(i[2])}' for i in clickstream_embed.columns]
    clickstream_embed.loc['0'] = np.empty(len(clickstream_embed.columns))

    dtype_clickstream = list()
    for x in clickstream_embed.dtypes.tolist():
        if x=='int64':
            dtype_clickstream.append('int16')
        elif(x=='float64'):
            dtype_clickstream.append('float32')
        else:
            dtype_clickstream.append('object')

    dtype_clickstream = dict(zip(clickstream_embed.columns.tolist(), dtype_clickstream))
    clickstream_embed = clickstream_embed.astype(dtype_clickstream)
    clickstream_embed.reset_index(drop=False, inplace=True)
    return clickstream_embed

In [8]:
%%time

df_click = pd.read_csv(f'data/clickstream.csv')
df_click = click_types(df_click)
df_click = click_pivot(df_click)

CPU times: user 1min 43s, sys: 16.3 s, total: 1min 59s
Wall time: 2min 7s


/tmp/ipykernel_206666/2608904884.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  clickstream_embed.reset_index(drop=False, inplace=True)


In [9]:
df_click_train = df_click[lambda x: x['user_id'].isin(df_matching_train['rtk'].values)]
df_click_valid = df_click[lambda x: x['user_id'].isin(df_matching_valid['rtk'].values)]
df_click_test = df_click[lambda x: x['user_id'].isin(df_matching_test['rtk'].values)]

print(df_click_train.shape, df_click_valid.shape, df_click_test.shape)

(9782, 403) (2446, 403) (2446, 403)


In [10]:
def trx_types(df):
    df['mcc_code'] = df['mcc_code'].astype(str)
    df['currency_rk'] = df['currency_rk'].astype(str)
    df['event_time'] = pd.to_datetime(df['transaction_dttm']).astype(int) / 1e9
    return df[['user_id', 'event_time', 'mcc_code', 'currency_rk', 'transaction_amt']]

def trx_pivot(df):
    bankclient_embed = df.pivot_table(index = 'user_id', 
                        values=['transaction_amt'],
                        columns=['mcc_code'],
                        aggfunc=['sum','mean', 'count']).fillna(0)
    bankclient_embed.columns = [f'{str(i[0])}-{str(i[2])}' for i in bankclient_embed.columns]
    
    dtype_bankclient = list()
    for x in bankclient_embed.dtypes.tolist():
        if x=='int64':
            dtype_bankclient.append('int16')
        elif(x=='float64'):
            dtype_bankclient.append('float32')
        else:
            dtype_bankclient.append('object')
    
    dtype_bankclient = dict(zip(bankclient_embed.columns.tolist(), dtype_bankclient))
    bankclient_embed = bankclient_embed.astype(dtype_bankclient)
    bankclient_embed.reset_index(drop=False, inplace=True)
    return bankclient_embed

In [11]:
%%time

df_trx = pd.read_csv(f'data/transactions.csv')
df_trx = trx_types(df_trx)
df_trx = trx_pivot(df_trx)

df_trx_train = df_trx[lambda x: x['user_id'].isin(df_matching_train['bank'].values)]
df_trx_valid = df_trx[lambda x: x['user_id'].isin(df_matching_valid['bank'].values)]
df_trx_test = df_trx[lambda x: x['user_id'].isin(df_matching_test['bank'].values)]

print(df_trx_train.shape, df_trx_valid.shape, df_trx_test.shape)

/tmp/ipykernel_206666/3274736351.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bankclient_embed.reset_index(drop=False, inplace=True)


(11721, 1159) (2930, 1159) (2930, 1159)
CPU times: user 28.4 s, sys: 3.68 s, total: 32.1 s
Wall time: 33.4 s


## Metric learning embeddings

In [12]:
transactions = pd.read_csv(f'data/transactions.csv')

print(transactions.shape)
print(transactions.user_id.nunique())

transactions.head()

(19821910, 5)
22533


,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,000932580e404dafbecd5916d4640938,5411,48,-361.07230,2020-08-03 08:05:23
1,000932580e404dafbecd5916d4640938,5499,48,-137.31398,2020-08-05 01:27:40
2,000932580e404dafbecd5916d4640938,5499,48,-138.84981,2020-08-05 03:28:11
3,000932580e404dafbecd5916d4640938,4829,48,-309.47653,2020-08-06 00:36:29
4,000932580e404dafbecd5916d4640938,5411,48,-133.47370,2020-08-09 00:30:13


In [13]:
from dltranz.data_preprocessing import PandasDataPreprocessor
from dltranz.seq_encoder import SequenceEncoder
from dltranz.models import Head
from dltranz.lightning_modules.emb_module import EmbModule
from dltranz.inference import get_embeddings

In [14]:
%%time
trx_preprocessor = PandasDataPreprocessor(
    col_id='user_id',
    cols_event_time='transaction_dttm',
    time_transformation='hours_from_min',
    cols_category=["mcc_code", "currency_rk"],
    cols_log_norm=["transaction_amt"],
)

trx_dataset = trx_preprocessor.fit_transform(transactions)

len(trx_dataset)

CPU times: user 37.5 s, sys: 3.57 s, total: 41 s
Wall time: 41.1 s


22533

In [15]:
seq_encoder = SequenceEncoder(
    category_features=trx_preprocessor.get_category_sizes(),
    numeric_features=['transaction_amt'],
    trx_embedding_noize=0.003
)
head = Head(input_size=seq_encoder.embedding_size, use_norm_encoder=True)
trx_model = EmbModule(seq_encoder=seq_encoder, head=head)

In [16]:
from dltranz.data_load.data_module.emb_data_module import EmbeddingTrainDataModule

dm = EmbeddingTrainDataModule(
    dataset=trx_dataset,
    pl_module=trx_model,
    min_seq_len=25,
    seq_split_strategy='SampleSlices',
    category_names = trx_model.seq_encoder.category_names,
    category_max_size = trx_model.seq_encoder.category_max_size,
    split_count=5,
    split_cnt_min=25,
    split_cnt_max=200,
    train_num_workers=16,
    train_batch_size=256,
    valid_num_workers=16,
    valid_batch_size=256
)

In [17]:
import torch
import pytorch_lightning as pl

import logging
# logging.getLogger("lightning").addHandler(logging.NullHandler())
# logging.getLogger("lightning").propagate = False

trainer = pl.Trainer(
    # progress_bar_refresh_rate=0,
    max_epochs=150,
    gpus=1 if torch.cuda.is_available() else 0
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [18]:
%%time

trainer.fit(trx_model, dm)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | _seq_encoder       | SequenceEncoder  | 846 K 
1 | _validation_metric | BatchRecallTopPL | 0     
2 | _head              | Head             | 0     
--------------------------------------------------------
846 K     Trainable params
0         Non-trainable params
846 K     Total params
3.386     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

CPU times: user 1h 49min 51s, sys: 16min 42s, total: 2h 6min 34s
Wall time: 1h 59min 24s


In [19]:
%%time

trx_embeds = get_embeddings(
    data=trx_dataset,
    model=trx_model, 
    category_names = trx_model.seq_encoder.category_names,
    category_max_size = trx_model.seq_encoder.category_max_size,
)

22533it [00:00, 22981.65it/s]
                                                                                                                                        

CPU times: user 34.1 s, sys: 13.3 s, total: 47.4 s
Wall time: 44.6 s


In [20]:
trx_embeds_df = pd.DataFrame(data=trx_embeds, columns=[f'embed_{i}' for i in range(trx_embeds.shape[1])])
trx_embeds_df['user_id'] = [x['user_id'] for x in trx_dataset]

trx_embeds_df.head()

,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_503,embed_504,embed_505,embed_506,embed_507,embed_508,embed_509,embed_510,embed_511,user_id
0,0.091485,-0.108873,0.013094,-0.974434,0.027217,-0.991314,-0.134523,-0.204748,-0.052441,-0.120273,...,0.168270,-0.088596,-0.097166,-0.692475,-0.266089,0.043477,-0.061554,-0.120555,0.989126,000932580e404dafbecd5916d4640938
1,-0.102059,0.132161,0.036850,-0.786810,-0.005040,-0.914710,-0.021032,-0.029007,-0.030026,-0.064974,...,-0.059505,-0.011978,-0.078693,-0.703464,0.327630,0.209434,-0.068717,-0.069901,0.908692,0009e36b42cb4caeb928a1e596819495
2,-0.248740,0.028777,0.037218,-0.839403,0.022314,-0.842904,0.123170,-0.347696,-0.019317,-0.070692,...,0.030039,0.096777,-0.062801,-0.764950,0.072273,-0.243047,-0.092489,-0.115780,0.924572,000b29acb6bd44f99473c1be5ca28f3c
3,0.049751,-0.029269,-0.237105,-0.908472,0.013908,-0.864875,0.415273,0.070509,-0.046024,-0.065222,...,-0.117661,0.029074,-0.083489,-0.659068,-0.231286,-0.246806,0.032731,-0.077745,0.949100,000c5327d99941fe934169838c65b92c
4,0.007169,0.148923,-0.054209,-0.849948,0.188024,-0.928779,0.121365,0.422354,-0.068434,-0.012294,...,0.056693,-0.098841,-0.071631,-0.659489,-0.001195,-0.261585,-0.123028,-0.047159,0.934825,000e0d54d7c945ebb8f6f855972e8396


In [21]:
df_trx_train = pd.merge(df_trx_train, trx_embeds_df, on='user_id', how='left')
df_trx_valid = pd.merge(df_trx_valid, trx_embeds_df, on='user_id', how='left')
df_trx_test = pd.merge(df_trx_test, trx_embeds_df, on='user_id', how='left')

### lightgbm model 

In [22]:
def prepare_dataset(df_matching, df_trx, df_click, neg_count=19):
    positive = pd.merge(df_matching, df_trx, left_on='bank', right_on='user_id').drop(columns='user_id')
    positive = pd.merge(positive, df_click, left_on='rtk', right_on='user_id').drop(columns='user_id')
    positive['target'] = 1

    rtks = df_click['user_id'].unique()

    negative = pd.DataFrame(data=df_trx['user_id'].values, columns=['bank'])
    negative['rtk'] = negative['bank'].apply(lambda x: np.random.choice(rtks, size=neg_count))
    negative = negative.explode('rtk')

    negative = pd.merge(negative, df_trx, left_on='bank', right_on='user_id').drop(columns='user_id')
    negative = pd.merge(negative, df_click, left_on='rtk', right_on='user_id').drop(columns='user_id')
    negative['target'] = 0

    dataset = pd.concat([positive, negative]).sample(frac=1)

    return dataset


train = prepare_dataset(df_matching_train, df_trx_train, df_click_train)
valid = prepare_dataset(df_matching_valid, df_trx_valid, df_click_valid)

train.shape, valid.shape

((234420, 2075), (58600, 2075))

In [31]:
from sklearn.metrics import roc_auc_score

import lightgbm as lgb

drop_columns = ['bank', 'rtk', 'target'] + [x for x in train.columns if x.startswith('sum') or x.startswith('mean') or x.startswith('count')]

CPU times: user 364 µs, sys: 56 µs, total: 420 µs
Wall time: 422 µs


In [32]:
%%time

TARGET = 'target'
CAT_FEATURES = []

params = dict(
    objective='binary',
    metric='auc',
    n_estimators=5000,
    boosting_type='gbdt',
    learning_rate=0.01,
    subsample=0.75,
    subsample_freq=1,
    feature_fraction=0.75,
    max_depth=8,
    lambda_l1=0.5,
    lambda_l2=0.5,
    num_leaves=128,
    random_state=42,
    verbose=-1
)

train_data = lgb.Dataset(train.drop(columns=drop_columns),
                                 label=train[TARGET],
                                 categorical_feature=CAT_FEATURES)

valid_data = lgb.Dataset(valid.drop(columns=drop_columns),
                         label=valid[TARGET],
                         categorical_feature=CAT_FEATURES)

lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data], verbose_eval=50)
preds = lgb_model.predict(valid.drop(columns=drop_columns))

metric_value = roc_auc_score(valid[TARGET], preds)
print(round(metric_value, 4))

/home/nikita/.local/share/virtualenvs/pytorch-lifestream-VR0LSEAW/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/nikita/.local/share/virtualenvs/pytorch-lifestream-VR0LSEAW/lib/python3.8/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[50]	training's auc: 0.844574	valid_1's auc: 0.69471
[100]	training's auc: 0.875435	valid_1's auc: 0.70674
[150]	training's auc: 0.896083	valid_1's auc: 0.715934
[200]	training's auc: 0.912544	valid_1's auc: 0.721987
[250]	training's auc: 0.924382	valid_1's auc: 0.726813
[300]	training's auc: 0.935556	valid_1's auc: 0.730088
[350]	training's auc: 0.944913	valid_1's auc: 0.733088
[400]	training's auc: 0.954174	valid_1's auc: 0.737374
[450]	training's auc: 0.961273	valid_1's auc: 0.740113
[500]	training's auc: 0.967675	valid_1's auc: 0.744369
[550]	training's auc: 0.972823	valid_1's auc: 0.747115
[600]	training's auc: 0.976875	valid_1's auc: 0.749951
[650]	training's auc: 0.980234	valid_1's auc: 0.751574
[700]	training's auc: 0.982842	valid_1's auc: 0.75308
[750]	training's auc: 0.985027	valid_1's auc: 0.75465
[800]	training's auc: 0.986965	valid_1's auc: 0.756153
[850]	training's auc: 0.988725	valid_1's auc: 0.758136
[900]	training's auc: 0.990194	valid_1's auc: 0.759054
[950]	training'

### Inference 

In [24]:
df_trx_valid.set_index('user_id', inplace=True)
df_click_valid.set_index('user_id', inplace=True)
df_matching_valid.set_index('bank', inplace=True)

In [25]:
def inference(df_trx, df_click, model, model_features, batch_size=100):

    list_of_rtk = list(df_click.index.unique())
    list_of_bank = list(df_trx.index.unique())

    submission = pd.DataFrame(list_of_bank, columns=['bank'])
    submission['rtk'] = submission['bank'].apply(lambda x: list_of_rtk)

    num_of_batches = int((len(list_of_bank))/batch_size)+1
    submission_ready = []

    for i in range(num_of_batches):
        bank_ids = list_of_bank[(i*batch_size):((i+1)*batch_size)]
        if len(bank_ids) != 0:
            part_of_submit = submission[submission['bank'].isin(bank_ids)].explode('rtk')
            part_of_submit = part_of_submit.merge(df_trx, how='left', left_on='bank', right_index=True
                                        ).merge(df_click, how='left', left_on='rtk', right_index=True).fillna(0)

            for feature in model_features:
                if feature not in part_of_submit.columns:
                    part_of_submit[i] = 0

            part_of_submit['predicts'] = model.predict(part_of_submit[model_features])
            part_of_submit = part_of_submit[['bank', 'rtk', 'predicts']]

            part_of_submit = part_of_submit.sort_values(by=['bank', 'predicts'], ascending=False).reset_index(drop=True)
            part_of_submit = part_of_submit.pivot_table(index='bank', values='rtk', aggfunc=list)
            part_of_submit['rtk'] = part_of_submit['rtk'].apply(lambda x: x[:100])
            part_of_submit['bank'] = part_of_submit.index
            part_of_submit = part_of_submit[['bank', 'rtk']]
            submission_ready.extend(part_of_submit.values)

    submission_final = np.array(submission_ready, dtype=object)
    return submission_final

In [33]:
%%time

preds = inference(df_trx_valid, df_click_valid, lgb_model, lgb_model.feature_name())

CPU times: user 1h 32min 13s, sys: 38.9 s, total: 1h 32min 52s
Wall time: 13min 4s


In [34]:
%%time

def score(preds, matching):
    pre = 0.0
    mrr = 0.0

    for ix_bank, pred in preds:
        match = matching.loc[ix_bank]['rtk']
        d = match in pred
        if d:
            pre += 1
            mrr += 1 / (1 + pred.index(match))

    pre /= len(preds)
    mrr /= len(preds)
    r1 = 2 * pre * mrr / (pre + mrr)

    return pre, mrr, r1

pre, mrr, r1 = score(preds, df_matching_valid)

print(f'pre: {pre:.3f}, mrr: {mrr:.3f}, r1: {r1:.3f}')

pre: 0.344, mrr: 0.174, r1: 0.231
CPU times: user 103 ms, sys: 0 ns, total: 103 ms
Wall time: 102 ms
